In [ ]:
import pandas as pd

Read Comuni

In [ ]:
df_comuni = pd.read_csv('classification/cls_comuni.csv', encoding="latin-1", sep=';')

In [ ]:
comuni_columns_to_keep = [
    'Codice Regione', 
    'Codice Provincia', 
    'Codice Comune formato alfanumerico', 
    'Denominazione in italiano',
    'Denominazione Regione']

# Mapping: vecchio nome -> nuovo nome
comuni_rename_map = {
    'Codice Comune formato alfanumerico': 'cod_com',
    'Denominazione in italiano': 'denominazione',
    'Codice Provincia': 'cod_prov',
    'Codice Regione': 'cod_reg',
    'Denominazione Regione': 'regione',
}

df_comuni = df_comuni[comuni_columns_to_keep].rename(columns=comuni_rename_map)

# Converti i nomi delle colonne in minuscolo
df_comuni.columns = df_comuni.columns.str.lower()

In [ ]:
df_comuni['regione'].unique()

Read RUNTS

In [ ]:
df_runts = pd.read_csv('classification/cls_runts.csv', 
                       encoding="latin-1", 
                       sep=';',  
                       dtype={"Codice_Provincia": str, "Codice_Regione": str})

In [ ]:
runts_columns_to_keep = [
    'Codice_Fiscale',
    'Repertorio', 
    'Denominazione',
    'Sezione',
    'Cognome',
    'Nome',
    'Rete',
    'Comune',
    'Codice_Provincia',
    'Provincia',
	'Codice_Regione',
    'Denominazione_Regione'
    ]

# Mapping: vecchio nome -> nuovo nome
runts_rename_map = {
    'Codice_Regione': 'cod_reg',
    'Codice_Provincia': 'cod_prov',
    'Denominazione_Regione': 'regione'
}

df_runts = df_runts[runts_columns_to_keep].rename(columns=runts_rename_map)

# Converti i nomi delle colonne in minuscolo
df_runts.columns = df_runts.columns.str.lower()


In [ ]:
df_runts.head(50).to_csv('runts_head.csv', index=False)

Read GINO microdata

In [ ]:
df_gino = pd.read_excel('protocolli_microdata.xlsx')

In [ ]:
df_gino.columns

In [ ]:
gino_columns_to_keep = [
    'ID_RISPONDENTE_RILEV',
    'PROGRESSIVO_QUEST',
    'REGIONE',
    'CODICE_CAV',
    'NOME_CAV',
    'TITOLO_DOC',
    'AMBITO',
    'SOGGETTI',
    'TIPI_SOGGETTI',
    'ATTORI',
    'MONITORAGGIO',
    'TIPO_RETE']

# Mapping: vecchio nome -> nuovo nome
gino_rename_map = {
    'TIPI_SOGGETTI': 'tipo_soggetto'
}

df_gino = df_gino[gino_columns_to_keep].rename(columns=gino_rename_map)

# Converti i nomi delle colonne in minuscolo
df_gino.columns = df_gino.columns.str.lower()

# Usando replace con regex: toglie "Regione" solo se è all'inizio
df_gino["regione"] = df_gino["regione"].str.replace(r"^Regione\s*", "", regex=True)

# Rimuove eventuali spazi bianchi iniziali e finali dalla colonna 'tipo_soggetto'
df_gino["tipo_soggetto"] = df_gino["tipo_soggetto"].str.strip()

REGOLA: Elimina dal dataset gino le regioni con una sola occorrenza

In [ ]:
# Conta le occorrenze di ogni regione
df_gino["regione_count"] = df_gino.groupby("regione")["regione"].transform("count")

# Filtra: tieni solo le righe dove la regione compare più di una volta
df_gino = df_gino[df_gino["regione_count"] > 1]

# (opzionale) elimina la colonna temporanea
df_gino = df_gino.drop(columns=["regione_count"])

REGOLA: Elimina le convenzioni

In [ ]:
# Porta tutto il campo 'titolo_doc' a minuscolo
df_gino["titolo_doc"] = df_gino["titolo_doc"].str.lower()

parole = ["convenzione", "coinvenzione", "conenzione"]
pattern = "|".join(parole)

df_gino = df_gino[~df_gino["titolo_doc"].str.contains(pattern, na=False, case=False, regex=True)]

In [ ]:
df_gino['regione'].value_counts()

In [ ]:
df_gino.head(50)

In [ ]:
df_gino['tipo_soggetto'].value_counts()

ATTORI: pre-processing  

In [ ]:
# Preprocessing e normalizzazione campo 'attori'
df_gino["attori"] = (
    df_gino["attori"]
    .fillna("")                               # sostituisce NaN con stringa vuota
    .str.replace(r"^\.$", "", regex=True)     # elimina i valori solo "."
    .str.replace(";", ",")                    # uniforma separatori
    .str.lower()                              # tutto minuscolo
    .str.strip()
)

# Esplode in più righe: un attore per riga
df_attori = (
    df_gino.assign(attore=df_gino["attori"].str.split(","))
      .explode("attore")
)

# Pulisce spazi e filtra vuoti
df_attori["attore"] = df_attori["attore"].str.strip()
df_attori = df_attori[df_attori["attore"] != ""].reset_index(drop=True)

# Ora hai un DataFrame con una colonna 'attore' pronta per il matching
df_attori.head(50)